# Oddments

:::{sidebar} Discovering Game Theory
This course uses an **online textbook**: [Game Theory, a Discovery Approach](https://nordstrommath.com/IntroGameTheory2e/intro_game_theory.html), by Jennifer Nordstrom.
:::

A very efficient method for solving $2\times 2$ matrix games with mixed strategy solutions (MSS) is using the technique of <span style="color:blue">**oddments**</span>. We calculate oddments by taking the absolute value differences across the rows and down the colums. From the oddments, we can calculate the proportions needed for $\vec r$ and $\vec c$ and then determine the overall game **value**.

:::{admonition} Teaching Example
Given the example game below, first verify that no domination exists for either player. Then, solve for Rose's and Colin's **optimal strategy mixture (OSM)**.

$$\begin{array}{cc}&\text{Colin}\\\text{Rose}&\begin{array}{r|rr}&A&B&\\ \hline A&7&2\\B&-1&6
\end{array}\end{array}$$
:::

To begin, we calculate the row oddments which are the absolute value difference of the entries in the row.

$$\begin{array}{ll}&\hspace{9mm}\text{Colin}\\\text{Rose}&\begin{array}{r|ccccccc}&A&B&&\text{Row Diffs}&\text{Oddments}\\\hline A&7&2&&|7-2|&5&\\B&-1&6&&|-1-6|&7
\end{array}\end{array}$$

Next, we calculate the column oddments in a similar way.

$$\begin{array}{ll}&\hspace{9mm}\text{Colin}\\\text{Rose}&\begin{array}{r|ccccccc}&A&B&&\text{Row Diffs}&\text{Oddments}\\\hline A&7&2&&|7-2|&5&\\B&-1&6&&|-1-6|&7\\\text{Column Diffs}&|7-(-1)|&|2-6|\\&8&4
\end{array}\end{array}$$

Creating the <span style="color:blue">**optimal strategy proportions (OSP)**</span> works as follows:

- OSP for Rose A is equal to $$\frac{\text{Row B Oddment}}{\text{Sum of Oddments}}$$
- OSP for Rose B is equal to $$\frac{\text{Row A Oddment}}{\text{Sum of Oddments}}$$

Notice that the numerators **swap position** compared to the oddments as demonstrated with arrows below.

$$\begin{array}{ll}&\hspace{9mm}\text{Colin}\\\text{Rose}&\begin{array}{r|ccccccc}&A&B&&\text{Row Diffs}&\text{Oddments}&&\text{Rose OSP}\\\hline A&7&2&&|7-2|&5&\searrow&\frac{7}{12}\\B&-1&6&&|-1-6|&7&\nearrow&\frac{5}{12}\\\text{Column Diffs}&|7-(-1)|&|2-6|&&\\\text{Oddments}&8&4&&&\end{array}\end{array}$$

Colin's OSP are calculated in an analagous way as shown below.

$$\begin{array}{ll}&\hspace{9mm}\text{Colin}\\\text{Rose}&\begin{array}{r|ccccccc}&A&B&&\text{Row Diffs}&\text{Oddments}&&\text{Rose OSP}\\\hline A&7&2&&|7-2|&5&\searrow&\frac{7}{12}\\B&-1&6&&|-1-6|&7&\nearrow&\frac{5}{12}\\\text{Column Diffs}&|7-(-1)|&|2-6|&&\\\text{Oddments}&8&4&&&\\&\searrow&\swarrow\\ \text{Colin OSP}&\frac{4}{12}&\frac{8}{12}
\end{array}\end{array}$$

Thus, we have from the OSP the following **optimal strategy sets** for Rose and Colin.

$$\vec r = \left[\begin{array}{rr} \frac{7}{12}\\ \frac{5}{12}\end{array}\right]\hspace{5mm}\text{and}\hspace{5mm}\vec c = \left[\begin{array}{rr}\frac{1}{3}\\ \frac{2}{3}\end{array}\right]$$

We find the game value by determining Rose's (or Colin's) expected value given the opponent plays her or his optimal strategy proprotions:

$$E(\text{Rose }A) = (7) \frac{1}{3} + (2) \frac{2}{3} =\frac{11}{3}$$

If we insert the game solver code, we can evaluate our solution to verify it. Remember that we enter the matrix game values as

`example_game = np.array([[7,2],[-1,6]])`

We first must initialize python:

In [30]:
## Do not change this cell, only execute it. 
## This cell initializes Python so that numpy and scipy packages are ready to use.

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np
import scipy as sp
from scipy.optimize import linprog

Next, the game solver code:

In [31]:
def solver(payoff_matrix):
    payoff_matrix = np.array(payoff_matrix)
    row_counts, col_counts = payoff_matrix.shape
# 1. Shift matrix to ensure all values are positive
# This ensures the game value V > 0
    shift = abs(payoff_matrix.min()) + 1
    A_shifted = payoff_matrix + shift
# 2. Solve for Row Player (Maximizer)
# We minimize 1/V = sum(p_i/V). Let x_i = p_i/V
# Objective: min (1 * x1 + 1 * x2 + ...)
    c_row = np.ones(row_counts)
# Constraints: A_shifted.T * x >= 1  =>  -A_shifted.T * x <= -1
    A_ub_row = -A_shifted.T
    b_ub_row = -np.ones(col_counts)
    res_row = linprog(c_row, A_ub=A_ub_row, b_ub=b_ub_row, method='highs')
# Game Value V = 1 / sum(x_i)
    v_shifted = 1 / res_row.fun
    row_strategy = res_row.x * v_shifted
# 3. Solve for Column Player (Minimizer) using the Dual
# Objective: max (1 * y1 + 1 * y2 + ...) => min (-1 * y1 - 1 * y2)
    c_col = -np.ones(col_counts)
# Constraints: A_shifted * y <= 1
    A_ub_col = A_shifted
    b_ub_col = np.ones(row_counts)
    res_col = linprog(c_col, A_ub=A_ub_col, b_ub=b_ub_col, method='highs')
    col_strategy = res_col.x * (1 / abs(res_col.fun))
# Final Game Value
    game_value = v_shifted - shift
    print(f"Game Value: {np.round(game_value,3)}")
    print(f"Rose Strategy: {np.round(row_strategy,3).tolist()}")
    print(f"Colin Strategy: {np.round(col_strategy,3).tolist()}")
    return None


In [32]:
example_game = np.array([[7,2],[-1,6]])

solver(example_game)

Game Value: 3.667
Rose Strategy: [0.583, 0.417]
Colin Strategy: [0.333, 0.667]


Hence, we have verified that our oddments solution above was indeed correct.